In [7]:
import os
import torch
import torchvision

In [8]:
#from PImage import PIM
#from Data import Labels

In [9]:
NUM_CLASSES = 4

### load the pretrained model

In [12]:
model = torchvision.models.alexnet(pretrained=True)

### make only the last layer trainable

In [ ]:
for idx, param in enumerate(model.parameters()):
    if idx < len(list(model.parameters()))-2:
        param.requires_grad = False

### change the last layer's output size as the number of classes

In [14]:
model.classifier[-1].out_features = NUM_CLASSES

### prepare the data

In [32]:
import os
from torchvision import transforms
import torchvision.datasets as datasets

IMG_DIR = '/home/cm/DigiPath/consROI'

In [33]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])}
#data_transforms = {
    #'val': transforms.Compose([
    #    transforms.Resize(256),
    #    transforms.CenterCrop(224),
    #    transforms.ToTensor(),
    #    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    #]),
#}

In [37]:
image_datasets = {x: datasets.ImageFolder(os.path.join(IMG_DIR, x),
                                          data_transforms[x]) for x in ['train']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True, num_workers=4)  
                                              for x in ['train']}

class_names = image_datasets['train'].classes
print(class_names)

['Atypical', 'DCIS', 'Invasive', 'UDH']
